In [1]:
#引入需要的包裝

import pyvisa
import time

### AFG31101

===============Function List=================

* Set weveform type
* Set frequency
* Set amplitude
* Set output state
* Set amplitude limit
  
若要串接其他儀器 一樣使用class 定義其他儀器的function

In [2]:
class AFG_31101():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to AFG_31101.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("AFG_31101 connection closed.")
        else:
            print("No instrument to close.")

    def set_waveform(self, shape):
        if self.instrument:
            self.instrument.write(f"SOURce1:FUNCtion {shape}")
            time.sleep(0.05)
            print(f"set excitation form be {shape}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_frequency(self, freq_amplitude, freq_unit):
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed {freq_amplitude} {freq_unit}")
            time.sleep(0.05)
            print(f"set excitation frequency be {freq_amplitude}{freq_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_frequency(self):
        if self.instrument:
            self.instrument.write(f"SOURce1:FREQuency:FIXed?")
            time.sleep(0.05)
            return self.instrument.read()
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def set_amplitude(self, amp_magnitude, amp_unit):
        if self.instrument:
            self.instrument.write(f"SOURce1:VOLTage:LEVel:IMMediate:AMPLitude {amp_magnitude}{amp_unit}")
            time.sleep(0.05)
            print(f"set excitation amplitude be {amp_magnitude}{amp_unit}")
        else:
            raise Exception("Instrument not connected. Please connect first.")
    
    def output_on(self):
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe ON")
            time.sleep(0.05)
            print(f"turn AFG31101 output on")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def output_off(self):
        if self.instrument:
            self.instrument.write(f"OUTPut1:STATe OFF")
            time.sleep(0.05)
            print(f"turn AFG31101 output off")
        else:
            raise Exception("Instrument not connected. Please connect first.")

### MSO64B

===============Function List=================

* Set A 
* Set B
* Set C

若要串接其他儀器 一樣使用class 定義其他儀器的function

In [3]:
class MSO_64B():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to instrument.")
        print(self.query_identity())

    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
         if self.instrument:
             return self.instrument.query("*IDN?")
         else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("Instrument connection closed.")
        else:
            print("No instrument to close.")

    def clear(self):
        if self.instrument:
            self.instrument.write(f':CLEAR')
            time.sleep(0.05)  # 在命令之間暫停0.5
            print(f"clear acquistions, measurements, and waveforms")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def add_measurement(self,meas,channel):
        if self.instrument:       #新增CH1、CH2、CH3的MEAN測量值
            source_CMD = f":DISplay:SELect:WAVEView1:SOUrce {channel}" #測量訊號源CMD
            type_CMD = f":MEASUREMENT:ADDMEAS {meas}" #測量類型CMD
            command = f"{source_CMD};{type_CMD}"
            self.instrument.write(command)
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"add measurent {meas} of {channel}")
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_population(self):
        if self.instrument:
            self.instrument.write(f'MEASUrement:MEAS4:RESUlts:ALLAcqs:POPUlation?')
            time.sleep(0.05)
            return int(self.instrument.read())
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def query_mean_value(self, meas):
        if self.instrument:        
            self.instrument.write(f'MEASUrement:{meas}:RESUlts:ALLAcqs:MEAN?')
            time.sleep(0.1)
            return self.instrument.read()
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def detect_clip(self,clip_check):
        if self.instrument:
            return self.instrument.query(f":{clip_check}:CLIPping?")
            time.sleep(0.05)  # 在命令之間暫停0.5秒
        else:
            raise Exception("Instrument not connected. Please connect first.")
            
    def query_vertical_scale(self,clip_check):
        if self.instrument:
            return self.instrument.query(f":{clip_check}:SCAle?")
            time.sleep(0.05)  # 在命令之間暫停0.5秒
        else:
            raise Exception("Instrument not connected. Please connect first.")
    
    def set_vertical_scale(self, clip_channel, new_vertical_value):
        if self.instrument:
            self.instrument.write(f":{clip_channel}:SCAle {new_vertical_value}")
            time.sleep(0.05)  # 在命令之間暫停0.5秒
            print(f"set {clip_channel} vertical scale be {new_vertical_value}")
        else:
            raise Exception("Instrument not connected. Please connect first.")
     

In [4]:
def wait_for_population(mso, threshold=20):
    while True: #重複查詢採樣次數(population)
        pop_num = mso.query_population() 
        print(f"Current population: {pop_num}") #列出每0.5秒的pop次數
        if pop_num >= threshold: #當pop次數大於等於100，結束while迴圈
            print("Poplution採樣數量已超過100") 
            break #跳脫迴圈
        time.sleep(2)  # 防止過於頻繁的查詢，調整每2秒查詢一次

In [5]:
def adjust_vertical_scale(mso):
    """
    調整示波器的垂直比例以防止剪切。
    :param mso: MSO 64B 示波器物件
    :param clip_channels: 需要檢測剪切的通道列表
    """
    clip_channels = ['CH1','CH2','CH3']#分別測試ch1、ch2、ch3是否clipping?
    
    for clip_channel in clip_channels:
        while True: #重複測試到最後沒通道clipping & vertical scale都設定到80%
            clip_result = float(mso.detect_clip(clip_channel)) 
            print(f"clipping state:{clip_result}") #顯示1代表有clipping，0代表沒有clipping
            if clip_result == 1: #clipping時
                clip_vertical_scale = float(mso.query_vertical_scale(clip_channel))
                new_clip_vertical_scale = clip_vertical_scale + 40e-3 #將當下的vertical scale以20mV/div為step加上去
                mso.set_vertical_scale(clip_channel, new_clip_vertical_scale) #設定新的vertical scale
                time.sleep(0.3) #最後調整到沒有clipping等待久一點，讓下面程序執行正常
                print(f"{clip_channel} is clipping, auto set vertical scale from {clip_vertical_scale} to {new_clip_vertical_scale}")
            elif clip_result == 0:
                if clip_channel == 'CH1': #當channel沒有clipping時
                    for meas in ['MEAS3']:
                        clip_amplitude = float(mso.query_mean_value(meas)) #看ch1的amplitude大小
                        print(clip_amplitude)
                        new_vertical_scale = clip_amplitude/8 #將amplitude大小除以8及為理想的80% vertical scale
                        mso.set_vertical_scale(clip_channel, new_vertical_scale) #設定新的vertical scale
                        print(f"{clip_channel} no clipping, auto set vertical scale to {new_vertical_scale}")
                        time.sleep(0.1)
                elif clip_channel == 'CH2':
                    for meas in ['MEAS4']:
                        clip_amplitude = float(mso.query_mean_value(meas))
                        print(clip_amplitude)
                        new_vertical_scale = clip_amplitude/8
                        mso.set_vertical_scale(clip_channel, new_vertical_scale)
                        print(f"{clip_channel} no clipping, auto set vertical scale to {new_vertical_scale}")
                        time.sleep(0.1)
                else:
                    for meas in ['MEAS5']:
                        clip_amplitude = float(mso.query_mean_value(meas))
                        print(clip_amplitude)
                        new_vertical_scale = clip_amplitude/8
                        mso.set_vertical_scale(clip_channel, new_vertical_scale)
                        print(f"{clip_channel} no clipping, auto set vertical scale to {new_vertical_scale}")
                        time.sleep(0.1)
                break

In [6]:
def main():
    
    # =========================Parmeters===============================
    #=====AFG_31101 parameter=====
    """
    shape:{SINusoid|SQUare|PULSe|RAMP|PRNoise|DC|SINC|GAUSsian|LORentz|ERISe|EDECay|HAVersine
     |EMEMory[1]|EMEMory2|EFILe} 
    amplitude:
        magnitude:output amplitude
        <units>::=[VPP | VRMS | DBM]
    frequency:
        magnitude:output frequency
        <units>::=[Hz | kHz | MHz]
    output_state:{ON|OFF}
    """
    #設置輸入源訊號參數
    shape = 'SINusoid'
    frequency = {'magnitude': '1','unit': 'MHz'}
    amplitude = {'magnitude': '1.8','unit': 'VPP'}
    output_state = "ON"

    #=====MSO64B parameter=====
    """
    
    """
    #針對display的通道新增測量項
    measurements = [
        {'channel': 'CH1','measurement': 'AMPLITUDE'},
        {'channel': 'CH2','measurement': 'AMPLITUDE'},
        {'channel': 'CH3','measurement': 'AMPLITUDE'},
    ]
    
    clip_channels = ['CH1','CH2','CH3']
    
    #====================Instrument Connect Test===========================
    #=====AFG_31101 connection=====
    AFG_instrument_address = 'USB0::0x0699::0x0359::C016504::INSTR'
    afg = AFG_31101(AFG_instrument_address)
    #====MSO 64B connection=====
    instrument_address = 'USB0::0x0699::0x0530::C048992::INSTR'
    mso = MSO_64B(instrument_address)

    
     #若有其他儀器串接下去...

    #============================Auto Test================================= 
    #=====AFG_31101 control=====
    afg.set_waveform(shape) #設置訊號類

    freq_magnitude = frequency['magnitude']
    freq_unit = frequency['unit']
    afg.set_frequency(freq_magnitude,freq_unit) #設置訊號頻率

    amp_magnitude = amplitude['magnitude']
    amp_unit = amplitude['unit']
    afg.set_amplitude(amp_magnitude,amp_unit) #設置訊號振幅

    if output_state == 'ON': #判斷書出狀態，選擇def
        afg.output_on()
    else:
        afg.output_off()

    #=====MSO 64B control=====
    # for measurement in measurements:
    #     channel = measurement['channel']
    #     meas = measurement['measurement']
    #     mso.add_measurement(meas,channel)
    
    mso.clear()
    time.sleep(0.5)

    wait_for_population(mso) #先等待pop到一定次數再抓數值
    
    # 調整vertical scale
    adjust_vertical_scale(mso)

    mso.clear() #將pop歸零
    time.sleep(0.5)
    wait_for_population(mso) #pop次數超過100繼續往下
    afg.output_off() #deskew & attenuation後將訊號供應AFG輸出關閉
        
    # ============================ Close Instrument=================================
    afg.close_instrument()
    mso.close_instrument()
    
if __name__ == "__main__":
    main()

Connected to AFG_31101.
TEKTRONIX,AFG31101,C016504,SCPI:99.0 FV:1.6.1

Connected to instrument.
TEKTRONIX,MSO64B,C048992,CF:91.1CT FV:1.44.3.433

set excitation form be SINusoid
set excitation frequency be 1MHz
set excitation amplitude be 1.8VPP
turn AFG31101 output on
clear acquistions, measurements, and waveforms
Current population: 1
Current population: 1
Current population: 1
Current population: 1
Current population: 39
Poplution採樣數量已超過100
clipping state:1.0
set CH1 vertical scale be 0.10200000000000001
CH1 is clipping, auto set vertical scale from 0.002 to 0.10200000000000001
clipping state:1.0
set CH1 vertical scale be 0.202
CH1 is clipping, auto set vertical scale from 0.102 to 0.202
clipping state:1.0
set CH1 vertical scale be 0.30200000000000005
CH1 is clipping, auto set vertical scale from 0.202 to 0.30200000000000005
clipping state:1.0
set CH1 vertical scale be 0.402
CH1 is clipping, auto set vertical scale from 0.302 to 0.402
clipping state:0.0
3.5395494510135
set CH1 verti